In [10]:
from pythreejs import *
from IPython.display import display
import sys, os, random, time
from math import *
import ode


# draw_body
def draw_body(body):
    """Draw an ODE body.
    """

    x,y,z = body.getPosition()
    R = body.getRotation()
    rot = [R[0], R[3], R[6], 0.,
           R[1], R[4], R[7], 0.,
           R[2], R[5], R[8], 0.,
           x, y, z, 1.0]
    if body.shape=="box":
        sx,sy,sz = body.boxsize
        body.widget = Mesh(geometry=BoxGeometry(width=sx,height=sy,depth=sz), 
                           material=LambertMaterial(color='red'), 
                           position=(x,y,z))
        body.widget.quaternion_from_rotation(R)

def create_box(world, space, density, lx, ly, lz):
    """Create a box body and its corresponding geom."""

    # Create body
    body = ode.Body(world)
    M = ode.Mass()
    M.setBox(density, lx, ly, lz)
    body.setMass(M)

    # Set parameters for drawing the body
    body.shape = "box"
    body.boxsize = (lx, ly, lz)

    # Create a box geom for collision detection
    geom = ode.GeomBox(space, lengths=body.boxsize)
    geom.setBody(body)

    return body, geom

# drop_object
def drop_object():
    """Drop an object into the scene."""

    global bodies, geom, counter, objcount

    body, geom = create_box(world, space, 1000, 1.0,0.2,0.2)
    body.setPosition( (random.gauss(0,0.1),3.0,random.gauss(0,0.1)) )
    theta = random.uniform(0,2*pi)
    ct = cos (theta)
    st = sin (theta)
    body.setRotation([ct, 0., -st, 0., 1., 0., st, 0., ct])
    bodies.append(body)
    geoms.append(geom)
    counter=0
    objcount+=1
    #print "drop position ",body.getPosition()
    #print "drop position --geom ",geom.getPosition()
    #print "drop quaternion",body.getQuaternion()

# Collision callback
def near_callback(args, geom1, geom2):
    """Callback function for the collide() method.

    This function checks if the given geoms do collide and
    creates contact joints if they do.
    """

    # Check if the objects do collide
    contacts = ode.collide(geom1, geom2)

    # Create contact joints
    world,contactgroup = args
    for c in contacts:
        c.setBounce(0.2)
        c.setMu(5000)
        j = ode.ContactJoint(world, contactgroup, c)
        j.attach(geom1.getBody(), geom2.getBody())



######################################################################


x = 0
y = 0
width = 640
height = 480

# Create a world object
world = ode.World()
world.setGravity( (0,-9.81,0) )
world.setERP(0.8)
world.setCFM(1E-5)

# Create a space object
space = ode.Space()

# Create a plane geom which prevent the objects from falling forever
floor = ode.GeomPlane(space, (0,1,0), -3)
# A list with ODE bodies
bodies = []

# The geoms for each of the bodies
geoms = []

# A joint group for the contact joints that are generated whenever
# two bodies collide
contactgroup = ode.JointGroup()

# Some variables used inside the simulation loop
fps = 20
dt = 1.0/fps
running = True
state = 0
counter = 0
objcount = 0
lasttime = time.time()

drop_object()
draw_body(bodies[0])

children=[b.widget for  b  in bodies]
children.append(AmbientLight(color=0x777777))
scene = Scene(children=children)
c = PerspectiveCamera(position=[7,0,7], up=[0,1,0], children=[DirectionalLight(color='white', 
                                                                             position=[3,5,1], 
                                                                             intensity=0.5)])
renderer = Renderer(camera=c, scene = scene, controls=[OrbitControls(controlling=c)])
ar = float(renderer.height)/(renderer.width)
renderer.width = 600
renderer.height = int(ar*renderer.width)
display(renderer)

lasttime = time.time()
for i in range(299):
    t = dt - (time.time() - lasttime)
    #drop a new bar  every 50 frames
    if (i+1)%50 == 0:
        drop_object()
        draw_body(bodies[-1])
        scene.children = [bodies[-1].widget] + scene.children
    #advance the simulation
    if (t > 0):
        time.sleep(t)
        space.collide((world,contactgroup), near_callback)
        # Simulation step
        world.step(dt)
        # Remove all contact joints
        contactgroup.empty()
        for b in bodies:
            x,y,z = b.getPosition()
            #print "current body position",x,y,z
            R = b.getRotation()
            if b.shape=="box":
                b.widget.position=(x,y,z)
                #print "current widget position",b.widget.position
                b.widget.quaternion_from_rotation(R)
    lasttime = time.time()